# xef.ai API Overview

In [1]:
@file:DependsOn("com.xebia:xef-core:0.0.5-alpha.119")
@file:DependsOn("com.xebia:xef-reasoning:0.0.5-alpha.119")
@file:DependsOn("com.xebia:xef-openai-client:0.0.5-alpha.119")
%use serialization

In [2]:
import com.xebia.functional.openai.generated.model.CreateChatCompletionRequestModel
import com.xebia.functional.xef.AIConfig
import com.xebia.functional.xef.Config
import kotlinx.coroutines.runBlocking

val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

val config = AIConfig(model = CreateChatCompletionRequestModel.gpt_4o_mini, config = Config(token = apiKey))

## Chat API

In [3]:
import com.xebia.functional.xef.AI

runBlocking {
    AI<String>("How do I create in Kotlin a plus function for a Point data class that adds two Points together in the form point1 + point2?", config = config)
}

To create a `plus` function for a `Point` data class in Kotlin that allows you to add two `Point` instances together using the `+` operator, you can define an operator function. Here's a simple example:

```kotlin
// Define the Point data class
data class Point(val x: Int, val y: Int) {
    // Define the operator function for addition
    operator fun plus(other: Point): Point {
        return Point(this.x + other.x, this.y + other.y)
    }
}

fun main() {
    val point1 = Point(1, 2)
    val point2 = Point(3, 4)
    val result = point1 + point2  // Using the plus operator
    println(result)  // Output: Point(x=4, y=6)
}
```

In this example:
- The `Point` data class has two properties: `x` and `y`.
- The `plus` operator function takes another `Point` as a parameter and returns a new `Point` that is the sum of the two points.
- In the `main` function, you can see how to use the `+` operator to add two `Point` instances.

## Prompts

In [4]:
import com.xebia.functional.xef.prompt.Prompt
import com.xebia.functional.xef.prompt.PromptBuilder.Companion.system
import com.xebia.functional.xef.prompt.PromptBuilder.Companion.user
import com.xebia.functional.xef.prompt.configuration.PromptConfiguration

runBlocking {
    AI<String>(Prompt(CreateChatCompletionRequestModel.gpt_4o_mini) {
        +system(
            """
                You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
                    - Use clean code principles.
                    - Write concise and readable APIs.
                    - Stick to idiomatic Kotlin conventions.
                    - Leverage higher-order functions, extensions, and null-safety features.
                    - Avoid unnecessary complexity; simplify the code when possible.
                    - Ensure the code is optimized for performance.
                    - Structure the code clearly and use best practices.
                    - Provide concise and to-the-point responses.
            """.trimIndent()
        )
        +user(
            """
                What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
                ```kotlin
                "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
                ```
            """.trimIndent()
        )
    }, config = config)

}

The `foldRight` function in Kotlin is a higher-order function that processes a collection (or in this case, a string) from the right to the left, applying a specified operation. It takes two parameters: an initial accumulator value and a lambda function that defines how to combine each element with the accumulator.

In the given example:
```kotlin
"baba".foldRight("") { x, acc -> "$x${acc}${acc}" }
```
- The initial accumulator is an empty string `""`.
- The lambda function takes each character `x` from the string and the current accumulator `acc`, and combines them into a new string by prepending `x` and appending `acc` twice.

Let's break it down step by step:

1. Start with the last character of the string, which is `a`, and the initial accumulator `""`:
   - Result: `"a" + "" + "" = "a"`

2. Move to the second last character, which is `b`, and the accumulator is now `"a"`:
   - Result: `"b" + "a" + "a" = "baaa"`

3. Move to the third character, which is `a`, and the accumulator is 

## Json_Mode

In [5]:
/**
 * Represents a person with basic personal attributes.
 *
 * @property name The name of the person.
 * @property age The age of the person in years.
 * @property height The height of the person in meters.
 * @property married Indicates if the person is married.
 */
@Serializable
data class Person(val name: String, val age: Int, val height: Double, val married: Boolean)

In [6]:
runBlocking {
    AI<Person>(Prompt(CreateChatCompletionRequestModel.gpt_4o_mini) {
        +user(
            """
        John is 42 years old and lives an independent life.
        He stands 1.75 meters tall and carries himself with confidence.
        Currently unmarried, he enjoys the freedom to focus on his personal goals and interests.
        """
        )
    }, config = config)
}

Person(name=John, age=42, height=1.75, married=false)

In [8]:
@Serializable
data class Step(val explanation: String, val output: String)
@Serializable
data class MathResponse(val steps: List<Step>, val finalAnswer: String)


runBlocking {
    val response = AI<MathResponse>(Prompt(CreateChatCompletionRequestModel.gpt_4o_mini) {
        +system("You are a helpful math tutor.")
        +user("solve 8x + 31 = 2")
    }, config = config)

    println(response.steps.withIndex().joinToString("\n") { "${it.index + 1}. ${it.value.explanation}" })
    println("Answer:")
    println(response.finalAnswer)
}

1. Subtract 31 from both sides to isolate the term with x.
2. Calculate the right side: 2 - 31 = -29.
3. Divide both sides by 8 to solve for x.
4. Simplify the fraction if necessary.
Answer:
x = -3.625
